# **LSTM - create your project report**

Srivarshan 19BAI1078

In [21]:
# importing libraries
import numpy
import sys
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [2]:
# download data
! rm /content/11-0.txt
! rm /content/wonderland.txt
! wget https://www.gutenberg.org/files/11/11-0.txt
! mv /content/11-0.txt /content/wonderland.txt

rm: cannot remove '/content/11-0.txt': No such file or directory
rm: cannot remove '/content/wonderland.txt': No such file or directory
--2021-11-23 04:49:28--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174313 (170K) [text/plain]
Saving to: ‘11-0.txt’

11-0.txt            100%[===================>] 170.23K  --.-KB/s    in 0.1s    

2021-11-23 04:49:28 (1.65 MB/s) - ‘11-0.txt’ saved [174313/174313]



In [3]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [4]:
# print example text
raw_text[:100]

'\ufeffthe project gutenberg ebook of alice’s adventures in wonderland, by lewis carroll\n\nthis ebook is fo'

In [5]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  164047
Total Vocab:  64


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163947


In [8]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

In [9]:
# normalize
X = X / float(n_vocab)

In [10]:
# one hot encode the output variable
y = to_categorical(dataY)

In [11]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
# fit the data
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1281/1281 [==============================] - ETA: 0s - loss: 3.0151
Epoch 00001: loss improved from inf to 3.01506, saving model to weights-improvement-01-3.0151.hdf5
1281/1281 [==============================] - 51s 36ms/step - loss: 3.0151
Epoch 2/20
1281/1281 [==============================] - ETA: 0s - loss: 2.8396
Epoch 00002: loss improved from 3.01506 to 2.83959, saving model to weights-improvement-02-2.8396.hdf5
1281/1281 [==============================] - 46s 36ms/step - loss: 2.8396
Epoch 3/20
1281/1281 [==============================] - ETA: 0s - loss: 2.7651
Epoch 00003: loss improved from 2.83959 to 2.76510, saving model to weights-improvement-03-2.7651.hdf5
1281/1281 [==============================] - 46s 36ms/step - loss: 2.7651
Epoch 4/20
1281/1281 [==============================] - ETA: 0s - loss: 2.6990
Epoch 00004: loss improved from 2.76510 to 2.69899, saving model to weights-improvement-04-2.6990.hdf5
1281/1281 [==============================] - 46s 36ms/

In [15]:
# load the network weights
filename = "/content/weights-improvement-19-2.1243.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
# create reverse mapping of int to char
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [19]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" eeping up
the conversation a little.

“’tis so,” said the duchess: “and the moral of that is—‘oh, ’t "


In [22]:
# create the report
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

u know.”

“i whnl y she mabc thet har head anoid and nooked an a lona of the word, 
“ho you dnn’t tooe to the soen i sean ” she mabc thit hnr heneled to the thre, “ho wou don’t keve to the then ”huh the whrt oi then ”huh the toote i shonld then to be io ano oo tie tooe ”huh the toot,”

“i whnh you would bo i seen ” she hact said, “or tes toe tame then  the had netel hoe the tooe oi the sooee an the sooee  a dat rath the mook ou tro th the wood, 
“he aourse ”huh the was i saal?” said the manch hare.

“ie aourse ”huh h shen to tee so toe thit ”ou,” said alice, 
“io you dnn’t know the wourd ”ou tooe ”ou to wei so tee then ”huh the was i saad?” said the corpouse the had not to the tooe, 
“h was a lint ii t aaied in a foor ”ith toe toile ” she said to herself, “ih aourse the kant of the tore of the sooee an the sooee  a dat rath the mook of the tar of the warte, and the waited to tee toae th the coere  and was no the wood  she had nete the tooe  she had netel here to the thnt hi a dond tan 